In [ ]:
import tensorflow as tf
from tensorflow import reshape
from tensorflow.keras import datasets, layers, models, metrics, Model
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import h5py

print(tf.__version__)

In [ ]:
hf = h5py.File('xy-Mg22.h5', 'r')

In [ ]:
list(hf.keys())

In [ ]:
hf['features'].shape

In [ ]:
hf['targets'].shape

In [ ]:
data = hf['features']
label = hf['targets']

In [ ]:
plt.imshow(data[4])
plt.xlabel(label[4])
plt.show()

In [ ]:
train_data = data[0:1500]
test_data = data[1500:2000]
train_label = label[0:1500]
test_label = label[1500:2000]

In [ ]:
image_width, image_length = 128, 128
image_shape = (128,128)

#Reshape data (https://www.machinecurve.com/index.php/2020/04/13/how-to-use-h5py-and-keras-to-train-with-data-from-hdf5-files/)
train_data = train_data.reshape((len(train_data), image_width, image_length))
test_data = test_data.reshape((len(test_data), image_width, image_length))

In [ ]:
#Normalize datasets
train_data = train_data/255.0
test_data = test_data/255.0

In [ ]:
#This callback will stop the training when there is no improvement in 
#the validation loss for [patience] consecutive epochs.

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
FCNN_model = tf.keras.Sequential()
FCNN_model.add(layers.Flatten(input_shape=(128,128)))
FCNN_model.add(layers.Dense(150, activation='relu'))
FCNN_model.add(layers.Dropout(0.7))
FCNN_model.add(layers.Dense(150, activation='relu'))
FCNN_model.add(layers.Dropout(0.7))
FCNN_model.add(layers.Dense(3, activation = 'softmax'))

In [ ]:
FCNN_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0000005), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=['accuracy'])

In [ ]:
FCNN_model.summary()

In [ ]:
FCNN_history = FCNN_model.fit(train_data, train_label,
                             epochs=40,
                             batch_size = 32,
                             shuffle = True,
                             callbacks = [callback],
                             validation_split = 0.3
                             )

In [ ]:
result = FCNN_model.evaluate(test_data, test_label, verbose=2)

In [ ]:
predictions = np.argmax(FCNN_model.predict(test_data[:]), axis=1)

In [ ]:
print(classification_report(test_label, predictions))

In [ ]:
print(FCNN_history.history.keys())

In [ ]:
plt.plot(FCNN_history.history['loss'])
plt.plot(FCNN_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#For confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools

In [ ]:
cm = confusion_matrix(test_label, predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                         normalize=False,
                         title='Confusion matrix',
                         include_values=True,
                         cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['1', '2']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion matrix')